In [1]:
import numpy as np
import tensorflow as tf

from sklearn.metrics import accuracy_score, classification_report

In [2]:
def one_hot_encoding(labels, num_classes):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    one_hot = np.zeros((num_labels, num_classes))
    one_hot.flat[index_offset + labels.ravel()] = 1
    return one_hot

def next_batch(data, target, last_index, 
               batch_size, train_data=True):
    start = last_index
    end = last_index + batch_size
    num_examples = data.shape[0]
    
    if start == 0 and train_data:
        perm = np.random.permutation(num_examples)
        data = data[perm]
        target = target[perm]
        
    if end > num_examples and train_data:
        perm = np.random.permutation(num_examples)
        data = data[perm]
        target = target[perm]
        start = 0
        end = batch_size
    elif end > num_examples:
        end = num_examples

    return end, data[start:end], target[start:end]

In [3]:
newsgroup = np.load('./resources/newsgroup.npz')

train_data = newsgroup['train_data']
train_target = newsgroup['train_target']
test_data = newsgroup['test_data']
test_target = newsgroup['test_target']
labels = newsgroup['labels']

batch_size = 100
input_size = train_data.shape[1]
num_classes = labels.shape[0]
hidden_layer_1 = 5000
hidden_layer_2 = 2000

x = tf.placeholder(tf.float32, [None, input_size])
y = tf.placeholder(tf.float32, [None, num_classes])

with tf.name_scope('hidden_layer_1'):
    W_h1 = tf.Variable(
               tf.truncated_normal(
                   [input_size, hidden_layer_1],
                   stddev=1.0 / np.sqrt(input_size)
               ),
               name='W_h1'
           )
    b_h1 = tf.Variable(
               tf.zeros([hidden_layer_1]),
               name='b_h1'
           )
    h1 = tf.nn.relu(tf.matmul(x, W_h1) + b_h1)

with tf.name_scope('hidden_layer_2'):
    W_h2 = tf.Variable(
               tf.truncated_normal(
                   [hidden_layer_1, hidden_layer_2],
                   stddev=1.0 / np.sqrt(hidden_layer_1)
               ),
               name='W_h2'
           )
    b_h2 = tf.Variable(
               tf.zeros([hidden_layer_2]),
               name='b_h2'
           )
    h2 = tf.nn.relu(tf.matmul(h1, W_h2) + b_h2)

with tf.name_scope('ouput_layer'):
    W_o = tf.Variable(
              tf.truncated_normal(
                  [hidden_layer_2, num_classes],
                  stddev=1.0 / np.sqrt(hidden_layer_2)
              ),
              name='W_o'
          )
    b_o = tf.Variable(
              tf.zeros([num_classes]),
              name='b_o'
          )
    logits = tf.matmul(h2, W_o) + b_o

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))

y_hat = tf.argmax(logits, 1)

train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

last_index = 0

for epoch in range(1, 2001):
    last_index, batch_data, batch_target =\
        next_batch(train_data, train_target,
                   last_index, batch_size)
    _, loss = sess.run([train_step, cross_entropy],
                       feed_dict={x: batch_data, y: one_hot_encoding(batch_target, num_classes)})
    if epoch % 100 == 0:
        print("Loss for epoch %02d: %.3f" % (epoch, loss))

Loss for epoch 100: 0.614
Loss for epoch 200: 0.057
Loss for epoch 300: 0.059
Loss for epoch 400: 0.413
Loss for epoch 500: 0.007
Loss for epoch 600: 0.019
Loss for epoch 700: 0.281
Loss for epoch 800: 0.004
Loss for epoch 900: 0.000
Loss for epoch 1000: 0.001
Loss for epoch 1100: 0.050
Loss for epoch 1200: 0.000
Loss for epoch 1300: 0.127
Loss for epoch 1400: 0.596
Loss for epoch 1500: 0.000
Loss for epoch 1600: 0.091
Loss for epoch 1700: 0.014
Loss for epoch 1800: 0.000
Loss for epoch 1900: 0.188
Loss for epoch 2000: 0.000


In [4]:
last_index = 0
test_examples = test_data.shape[0]
predictions = []

for _ in range(np.int(test_examples / batch_size) + 1):
    last_index, batch_data, _ = next_batch(test_data, test_target, last_index, batch_size, False)
    predictions.append(sess.run(y_hat, feed_dict={x: batch_data}))

predictions = np.concatenate(predictions)
accuracy = accuracy_score(test_target, predictions)

print("Accuracy: %.2f" % accuracy)

print(classification_report(test_target, predictions))

Accuracy: 0.87
             precision    recall  f1-score   support

          0       0.94      0.89      0.92       160
          1       0.77      0.78      0.78       195
          2       0.81      0.84      0.82       197
          3       0.82      0.72      0.77       196
          4       0.83      0.83      0.83       192
          5       0.86      0.89      0.87       196
          6       0.73      0.79      0.76       194
          7       0.94      0.81      0.87       198
          8       0.89      0.94      0.92       199
          9       0.94      0.94      0.94       199
         10       0.96      0.94      0.95       200
         11       0.99      0.90      0.94       198
         12       0.85      0.83      0.84       196
         13       0.92      0.91      0.92       198
         14       0.95      0.94      0.95       197
         15       0.89      0.92      0.90       200
         16       0.88      0.90      0.89       182
         17       0.96      0.